In [ ]:
!pip install python-binance
!pip install talib-binary
!pip install ta 

In [ ]:
import pandas as pd
import time
from binance import BinanceSocketManager,Client
import numpy
from pandas.core import frame
import nest_asyncio
import talib as ta
import sys

In [ ]:
#entrada na API
client = Client("N1rc9N8fYRn1Oyb9yoKIGwUIf3tUwVBxI5EyhkpC1blxeFG451qoCfMVNO0ES6Kl", "DZXKpom2aQ0lL0r9r9vXufvDBsRXV5vkP1L9IxsbQ5wg9CMJLSzJzTMLltQ80Yfo")

In [ ]:
#pega dados da binance 
def get_minute_data(ticker, interval, lookback):
      dados =  pd.DataFrame(client.get_historical_klines(ticker, interval , lookback +' ago UTC'))
      dados = dados.iloc[:,:6]
      dados.columns = ['Time','Open','High','Low','Close','Volume']
      dados = dados.set_index('Time')
      dados.index = pd.to_datetime(dados.index, unit='ms')
      dados = dados.astype(float)
    
      return dados

In [ ]:
from ta.trend import ADXIndicator


In [ ]:
#calcula adx
def get_adx(dados):
  adxI = ADXIndicator(dados['High'],dados['Low'],dados['Close'],8,False)
  dados['pos_directional_indicator'] = adxI.adx_pos()
  dados['neg_directional_indicator'] = adxI.adx_neg()
  dados['adx'] = adxI.adx()
  
  return dados



In [ ]:
#calcula as medias e verifica a agulhada
def mediasdidi(dados, tempo, ticker):
    sma20 = ta.SMA(dados['Close'], timeperiod= 20)
    sma8 = ta.SMA(dados['Close'], timeperiod = 8)
    sma3 = ta.SMA(dados['Close'], timeperiod= 3)
    situacao = '0'
    if (sma3[-1] > sma8[-1] and sma3[-1] > dados.Open[-1] and sma3[-1] < dados.Close[-1]) and (sma8[-1] > sma20[-1] and sma8[-1] > dados.Open[-1] and sma8[-1] < dados.Close[-1]) and (sma20[-1] > dados.Open[-1] and sma20[-1] < dados.Close[-1]):
      get_adx(dados)
      if dados['adx'][-1] > 25:
        print(f'possivel agulhada de compra no timeframe de {tempo} em {ticker} ADX >25')
      else:
        print(f'possivel agulhada de compra timeframe de {tempo} em {ticker} porém ADX < 25')
    elif (sma3[-1] < sma8[-1] and sma3[-1] < dados.Open[-1] and sma3[-1] > dados.Close[-1]) and (sma8[-1] < sma20[-1] and sma8[-1] < dados.Open[-1] and sma8[-1] > dados.Close[-1]) and (sma20[-1] < dados.Open[-1] and sma20[-1] > dados.Close[-1]):
      get_adx(dados)
      if dados['adx'][-1] > 25:
        print(f'possivel agulhada de venda no timeframe de {tempo} em {ticker} ADX > 25')
      else:
        print(f'possivel agulhada de venda timeframe de {tempo} em {ticker} porém ADX < 25')
    

In [ ]:

#lista de ativos
lista = ['BTCUSDT','VGXUSDT','WINUSDT','XECUSDT','YFIUSDT','SOLUSDT', 'ETHUSDT', 'BNBUSDT','COTIUSDT','ALICEUSDT','ANYUSDT','AVAXUSDT','CAKEUSDT',
         'CKBUSDT','CRVUSDT','MINAUSDT','MDTUSDT','KP3RUSDT','AAVEUSDT','AMPUSDT','ARUSDT','SUSHIUSDT','SCRTUSDT','REEFUSDT','MBOXUSDT','LUNAUSDT',
         'NEOUSDT','UNIUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'BNTUSDT','NEARUSDT','DYDXUSDT',
         'ICXUSDT', 'NULSUSDT', 'VETUSDT', 'LINKUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'DOTUSDT','EGLDUSDT','FILUSDT','FLOWUSDT','GALAUSDT','GRTUSDT',
         'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'CELRUSDT','DASHUSDT', 'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MATICUSDT', 'ATOMUSDT','CELOUSDT','TFUELUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT',  'DOGEUSDT', 'ANKRUSDT','COMPUSDT','TOMOUSDT', 'PERLUSDT', 'DENTUSDT', 'MFTUSDT', 'KEYUSDT', 'DOCKUSDT',  'CHZUSDT', 'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RVNUSDT',
         'HBARUSDT','STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'BCHUSDT', 'FTTUSDT', 'OGNUSDT','WAVESUSDT','ZENUSDT']



In [ ]:
lista2 =  ['OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MATICUSDT', 'ATOMUSDT','CELOUSDT','TFUELUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT',  'DOGEUSDT', 'ANKRUSDT','COMPUSDT','TOMOUSDT', 'PERLUSDT', 'DENTUSDT', 'MFTUSDT', 'KEYUSDT', 'DOCKUSDT',  'CHZUSDT', 'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RVNUSDT',
         'HBARUSDT','STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'BCHUSDT', 'FTTUSDT', 'OGNUSDT','WAVESUSDT','ZENUSDT']


In [ ]:
def main(asset):
  for i in asset:
     assetone = i
     frame1d = get_minute_data(assetone, '1d','50 days')
     frame2h = get_minute_data(assetone, '2h', '100 hours ')
     frame1h = get_minute_data(assetone, '1h', '50 hours ')
     frame4h = get_minute_data(assetone, '4h', '200 hours')
     frame15m = get_minute_data(assetone, '15m' , '500 min')
     print(f'analisando {i}')
     mediasdidi(frame1d, '1d', i)
     time.sleep(2)
     mediasdidi(frame1d, '2h', i)
     time.sleep(1)
     mediasdidi(frame1h,'1h', i)
     time.sleep(2)
     mediasdidi(frame4h, '4h', i)
     time.sleep(1)
     mediasdidi(frame15m, '15', i)


In [ ]:
#Rodando o scanning
while True:
    main(lista)
    print('reiniciando')
    time.sleep(2)

In [ ]:
     assetone = 'BTCUSDT'
     i = 'BTCUSDT'
     frame1d = get_minute_data(assetone, '1d','50 days')
     frame1h = get_minute_data(assetone, '1h', '50 hours ')
     frame4h = get_minute_data(assetone, '4h', '200 hours')
     frame15m = get_minute_data(assetone, '30m' , '900 min')
     pt = get_adx(frame15m)
     print(pt)
     print(pt['adx'][-1])
     print(f'analisando {i}')
     mediasdidi(frame1d, '1d', i)
     time.sleep(1)
     mediasdidi(frame1h,'1h', i)
     time.sleep(1)
     mediasdidi(frame4h, '4h', i)
     time.sleep(5)
     mediasdidi(frame15m, '15', i)


In [ ]:
  get_adx(dx)

